In [19]:
import os
import cv2
import numpy as np

In [20]:
def get_limits(color):
    c = np.uint8([[color]]) # here insert the bgr values which you want to convert to hsv
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    lowerLimit = hsvC[0][0][0], 40, 0
    upperLimit = hsvC[0][0][0], 255, 255

    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [21]:
def for_gray_hand():
    lowerLimit = 0, 0, 5
    upperLimit = 0, 0, 255

    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [22]:
def find_mask(mask, cls_num, output_dir, j):
    H, W = mask.shape
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    polygon = []
    # convert the contours to polygons
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            for point in cnt:
                x, y = point[0]
                polygon.append(x / W)
                polygon.append(y / H)

    # print the polygons
    with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'a') as f:
        for p_, p in enumerate(polygon):
            if p_ == (len(polygon) - 1):
                f.write('{}\n'.format(p))
            elif p_ == 0:
                f.write(str(cls_num) + ' {} '.format(p))
            else:
                f.write('{} '.format(p))
        f.close()

In [23]:
# in BGR colorspace
yellow = [0, 255, 255]
red = [0, 0, 255]
green = [0, 255, 0]
blue = [255, 0, 0]
gray = [216, 216, 216]

# image_path = os.path.join('./', 'couple_bg_01.jpg')
# load the binary mask and get its contours
mask = cv2.imread('./frame_multiple_rgby_01_25.png')
hsvMask = cv2.cvtColor(mask, cv2.COLOR_BGR2HSV)

lowerLimit_yellow, upperLimit_yellow = get_limits(yellow)
lowerLimit_red, upperLimit_red = get_limits(red)
lowerLimit_green, upperLimit_green = get_limits(green)
lowerLimit_blue, upperLimit_blue = get_limits(blue)
lowerLimit_gray, upperLimit_gray = for_gray_hand()


mask_yellow = cv2.inRange(hsvMask, lowerLimit_yellow, upperLimit_yellow)
mask_red = cv2.inRange(hsvMask, lowerLimit_red, upperLimit_red)
mask_green = cv2.inRange(hsvMask, lowerLimit_green, upperLimit_green)
mask_blue = cv2.inRange(hsvMask, lowerLimit_blue, upperLimit_blue)
mask_gray = cv2.inRange(hsvMask, lowerLimit_gray, upperLimit_gray)

cv2.imshow('red', mask_red)
cv2.imshow('blue', mask_blue)
cv2.imshow('green', mask_green)
cv2.imshow('yellow', mask_yellow)
cv2.imshow('gray', mask_gray)
cv2.waitKey(0)

if np.sum(mask_red == 255) != 0:
    print(np.sum(mask_red == 255), 0)
    find_mask(mask_red, 0, './', 'label0.txt')

if np.sum(mask_blue == 255) != 0:
    print(np.sum(mask_blue == 255), 1)
    find_mask(mask_blue, 1, './', 'label1.txt')

if np.sum(mask_green == 255) != 0:
    print(np.sum(mask_green == 255), 2)
    find_mask(mask_green, 2, './', 'label2.txt')

if np.sum(mask_yellow == 255) != 0:
    print(np.sum(mask_yellow == 255), 3)
    find_mask(mask_yellow, 3, './', 'label3.txt')

if np.sum(mask_gray == 255) != 0:
    print(np.sum(mask_gray == 255), 4)
    find_mask(mask_gray, 4, './', 'label4.txt')

cv2.destroyAllWindows()

12084 0
13716 1
12356 2
12946 3
66332 4
